<h1> Introduction </h1>

<h1> Theory </h1>

<h1> Methods </h1>

<h1> Code Implementation </h1>

In [ ]:
# read in data

<h2> Data Selection </h2>

<h2> Model Implementation </h2>

<h2> Hyper Parameter Tuning </h2>

<h2> Getting Final Results </h2>

<h1> Results </h1>



<h1> Conclusions </h1>






<h1> References </h1>
